## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


In [2]:
model_architecture = "PPO" # ADD YOUR MODEL NAME HERE
env_id = "LunarLander-v2" # ADD YOUR ENVIRONMENT NAME HERE
username = "PathFinderKR" # ADD YOUR USERNAME HERE
repo_id = f"{username}/{model_architecture}-{env_id}" # repository id
fps = 30 # frames per second

## Imports

In [3]:
# video
import imageio

# Hugging Face
from huggingface_sb3 import package_to_hub

# gymnasium
import gymnasium as gym

# stable-baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

## Before Training

In [4]:
# Create a LunarLander environment
env = gym.make("LunarLander-v2", render_mode="rgb_array")

print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())  # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())  # Take a random action

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [ 77.62832    -73.02351      4.9960732   -4.931017    -1.682841
   2.1771123    0.78808206   0.1734814 ]

 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 2


In [5]:
# Reset the environment
observation, info = env.reset()

frames = []
for _ in range(1000):
    frames.append(env.render()) # Render the environment
    
    # Take a random actions
    action = env.action_space.sample()
    #print(f"Action: {action}")
    
    # Do this action in the environment and get
    # next_state, reward, terminated, truncated and info
    observation, reward, terminated, truncated, info = env.step(action)
    #print(f"Reward: {reward}")

    # If the game is terminated (in our case we land, crashed) or truncated (timeout)
    if terminated or truncated:
        # Reset the environment
        #print("Game terminated. Resetting the environment")
        observation, info = env.reset()

env.close()

video_path = f"{env_id}-sample.mp4"
imageio.mimsave(video_path, frames, fps=fps)  # Save frames as video

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x6d17040] Warning: data is not aligned! This can lead to a speed loss


## Hyperparameters

In [6]:
policy = "MlpPolicy" # Policy model to use (MlpPolicy, CnnPolicy, ...)
n_steps = 1024 # Number of steps to use in each environment
batch_size = 128 # Number of experiences to use for each training step
n_epochs = 4 # Number of times to go through the data for each training step
gamma = 0.999 # Discount factor
gae_lambda = 0.98 # Lambda for Generalized Advantage Estimation (GAE)
ent_coef = 0.01 # Entropy coefficient for the loss calculation
verbose = 1 # Verbosity level

total_timesteps = 100000 # Total number of timesteps to train the model

## Environment

In [7]:
# Create a LunarLander environment
env = gym.make("LunarLander-v2", render_mode="rgb_array")

print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())  # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())  # Take a random action

# Reset the environment
observation, info = env.reset()

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [ 8.4423080e+01  7.6508369e+01  2.5650239e+00 -3.2432632e+00
 -1.8902570e+00 -4.6551008e+00  6.6383041e-02  3.6948079e-01]

 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 2


## Agent

In [8]:
model = PPO(
    policy=policy,
    env=env,
    n_steps=n_steps,
    batch_size=batch_size,
    n_epochs=n_epochs,
    gamma=gamma,
    gae_lambda=gae_lambda,
    ent_coef=ent_coef,
    verbose=verbose
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## Training

In [9]:
# Train it for 1,000,000 timesteps
model.learn(total_timesteps=total_timesteps)
# Save the model
model.save(f"{model_architecture}-{env_id}")

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -222     |
| time/              |          |
|    fps             | 642      |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 1024     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 99.9         |
|    ep_rew_mean          | -221         |
| time/                   |              |
|    fps                  | 612          |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0007524362 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.39        |
|    explained_variance   | -0.00214     |
|    learning_r

## Evaluation

In [10]:
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-200.31 +/- 31.180075770106562


## Video

In [11]:
# Reset the environment
observation, info = env.reset()

# Record the video
frames = []
for _ in range(1000):
    frames.append(env.render()) # Render the environment
    
    # Predict the action
    action, _ = model.predict(observation, deterministic=True)
    #print(f"Action: {action}")
    
    # Do the action in the environment and get 
    # next state, reward, terminated, truncated and info
    observation, reward, terminated, truncated, info = env.step(action)
    #print(f"Reward: {reward}")
    
    if terminated or truncated:
        observation, info = env.reset()
        
env.close()

# Save the video
video_path = f"{model_architecture}-{env_id}.mp4"
imageio.mimsave(video_path, frames, fps=fps)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x5c95040] Warning: data is not aligned! This can lead to a speed loss


## Push to Hub

In [12]:
## Define the commit message
commit_message = "Upload PPO LunarLander-v2 trained agent"

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))])

# PLACE the package_to_hub function you've just filled here
package_to_hub(
    model=model,  # Our trained model
    model_name=f"{model_architecture}-{env_id}",  # The name of our trained model
    model_architecture=model_architecture,  # The model architecture
    env_id=env_id,  # Name of the environment
    eval_env=eval_env,  # Evaluation Environment
    repo_id=repo_id,  # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name}
    commit_message=commit_message,
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpvr0eaz4p/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpvr0eaz4p/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpvr0eaz4p/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpvr0eaz4p/-step-0-to-step-1000.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

ℹ Pushing repo PathFinderKR/PPO-LunarLander-v2 to the Hugging Face
Hub


pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

...

PPO-LunarLander-v2.zip:   0%|          | 0.00/148k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

policy.optimizer.pth:   0%|          | 0.00/88.4k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/PathFinderKR/PPO-LunarLander-v2/tree/main/


CommitInfo(commit_url='https://huggingface.co/PathFinderKR/PPO-LunarLander-v2/commit/a4f2b2b617d051587e0ca7e0a0608eef06dbb5fe', commit_message='Upload PPO LunarLander-v2 trained agent', commit_description='', oid='a4f2b2b617d051587e0ca7e0a0608eef06dbb5fe', pr_url=None, pr_revision=None, pr_num=None)